In [1]:
import pandas as pd
import numpy as np
import regex as re
import warnings
warnings.filterwarnings('ignore')

In [2]:
actor = pd.read_csv('../data/actor.csv', encoding='latin1')
a=actor.copy()
category = pd.read_csv('../data/category.csv', encoding='latin1')
c=category.copy()
film = pd.read_csv('../data/film.csv', encoding='latin1')
f=film.copy()
i = pd.read_csv('../data/inventory.csv', encoding='latin1')
inventory=i.copy()
language = pd.read_csv('../data/language.csv', encoding='latin1')
l=language.copy()
old_HDD = pd.read_csv('../data/old_HDD.csv', encoding='latin1')
o=old_HDD.copy()
rental = pd.read_csv('../data/rental.csv', encoding='latin1')
r=rental.copy()

## Exploración archivos y limpieza previa.

In [3]:
#1º. Actores.

a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [4]:
a.isnull().sum() #No hay ningun valor nulo

actor_id       0
first_name     0
last_name      0
last_update    0
dtype: int64

In [5]:
a.columns #vamos a ponerlo bonito

Index(['actor_id', 'first_name', 'last_name', 'last_update'], dtype='object')

In [6]:
a.columns=a.columns.str.replace('_',' ').str.title()
a.columns

Index(['Actor Id', 'First Name', 'Last Name', 'Last Update'], dtype='object')

In [7]:
a['Last Update'].unique() #toda la columna es igual y no nos da ninguna información relevante, nos la cargamos 🆑

array(['2006-02-15 04:34:33'], dtype=object)

In [8]:
a.drop(columns=['Last Update'], inplace=True)
a.columns

Index(['Actor Id', 'First Name', 'Last Name'], dtype='object')

In [9]:
a.columns.duplicated().any() #sin duplicados

False

In [10]:
a[a.duplicated(subset=['First Name', 'Last Name'],keep=False)] # busco duplicados pero me centro solo en los nombes ya que los id son diferentes para cada uno,
                                                               # hay un actor repetido, vamos a quitarlo. El keep False lo saque del chat 🤫, porque si no
                                                               #lo ponía solo me mostraba el 109 con el nombre y no sabia de donde venía el duplicado.

,Actor Id,First Name,Last Name
100,101,SUSAN,DAVIS
109,110,SUSAN,DAVIS


In [11]:
a.drop([109],axis=0,inplace=True) # no me va a interferir porque no hay ninguna tabla que me relacione su id con nada, no peta 💣

In [12]:
a.reset_index(inplace=True,drop=True)
nuevos_id = pd.Series(range(1, len(a)+1)) # cambio los id para que no tener salto y reseteo index
a['Actor Id']=nuevos_id

a.iloc[107:112] # compruebo que no me pegue el salto, he hecho todo esto porque para el análisis me puede servir

,Actor Id,First Name,Last Name
107,108,WARREN,NOLTE
108,109,SYLVESTER,DERN
109,110,CAMERON,ZELLWEGER
110,111,RUSSELL,BACALL
111,112,MORGAN,HOPKINS


In [13]:
#2º) Categoría: 

c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category_id  16 non-null     int64 
 1   name         16 non-null     object
 2   last_update  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 512.0+ bytes


In [14]:
c.head()

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [15]:
c.isnull().sum() #sin nulos


category_id    0
name           0
last_update    0
dtype: int64

In [16]:
c.columns=c.columns.str.replace('_',' ').str.title() #ponemos bonitas las columnas

In [17]:
c.drop(columns=['Last Update'], inplace=True) #quitamos las update, no me vale
c.columns

Index(['Category Id', 'Name'], dtype='object')

In [31]:
c.duplicated().sum() #sin duplicados

0

In [20]:
#3º) Film:
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  0 non-null      float64
 6   rental_duration       1000 non-null   int64  
 7   rental_rate           1000 non-null   float64
 8   length                1000 non-null   int64  
 9   replacement_cost      1000 non-null   float64
 10  rating                1000 non-null   object 
 11  special_features      1000 non-null   object 
 12  last_update           1000 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 101.7+ KB


In [22]:
f.columns=f.columns.str.replace('_',' ').str.title()
f.columns

Index(['Film Id', 'Title', 'Description', 'Release Year', 'Language Id',
       'Original Language Id', 'Rental Duration', 'Rental Rate', 'Length',
       'Replacement Cost', 'Rating', 'Special Features', 'Last Update'],
      dtype='object')

In [32]:
f.isna().sum() 

Film Id                    0
Title                      0
Description                0
Release Year               0
Language Id                0
Original Language Id    1000
Rental Duration            0
Rental Rate                0
Length                     0
Replacement Cost           0
Rating                     0
Special Features           0
Last Update                0
dtype: int64